**【概要】**

課題は、特別なチームのプレーに対応するプレーヤーの追跡データから、実用的で実用的で斬新な洞察を生み出すことです。
Your challenge is to generate actionable, practical, and novel insights from player tracking data that corresponds to special teams play.

これらには以下が含まれますが、これらに限定されません。

1. 新しい特別チームの指標を作成します。2020年のビッグデータボウルで優勝したアルゴリズムは、NFL / NFLネットワークでオンエア配信に採用されており、今年の大会で得られる可能性のある特別チームのプレーの新しい統計が得られることを期待しています。
Create a new special teams metric. The winning algorithm from the 2020 Big Data Bowl has been adopted by the NFL/NFL Network for on air distribution, and we are hopeful that there could be a new stat for special teams plays that could come from this year’s competition

2. 特別チームの戦略を定量化します。特別チームのコーチは、リーグで最も創造的で革新的な選手の1人です。各チームのゲームの計画を比較/対比します。どの戦略が最良の結果をもたらしますか？採用できる他の戦略は何ですか？
Quantify special teams strategy. Special teams’ coaches are among the most creative and innovative in the league. Compare/contrast how each team game plans. Which strategies yield the best results? What are other strategies that could be adopted?


3. 特別チームの選手をランク付けします。各チームは、さまざまなプレーヤー（ロングスナッパー、キッカー、パンター、およびその他のユーティリティ特別チームプレーヤーを含む）を採用しています。それらはどのように相互に積み重なっていますか？
Rank special teams players. Each team employs a variety of players (including longsnappers, kickers, punters, and other utility special teams players). How do they stack up with respect to one another?


上記のリストは包括的ではなく、参加者がカバーするためのガイドとなることを意図したものでもありません。今年のコンテストでは、すべての特別チーム関連のアイデアを受け入れます。

1つのアイデアをより徹底的に検討する提出は、複数のアイデアをある程度徹底的に検討する提出よりも優先されます。

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns 
import matplotlib as mpl
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pylab as plt


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### 【Games Data】

ゲームデータ：にgames.csvは、各ゲームでプレーしているチームが含まれています。キー変数ですgameId。
ゲームデータ gameId：ゲーム識別子、一意（数値）
* season：ゲームのシーズン
* week：ゲームの週
* gameDate：ゲームの日付（時間、mm / dd / yyyy）
* gameTimeEastern：ゲームの開始時間（時間、HH：MM：SS、EST）
* homeTeamAbbr：ホームチームの3文字のコード（テキスト）
* visitorTeamAbbr：訪問チームの3文字のコード（テキスト）

In [ ]:
df_gamedata=pd.read_csv('../input/nfl-big-data-bowl-2022/games.csv')
df_gamedata.head()

In [ ]:
len(df_gamedata)

#### ■search correlated columns in games.csv.

In [ ]:
df_gamedata.corr()

In [ ]:
pp_games = sns.pairplot(df_gamedata)
plt.show()

In [ ]:
df=df_gamedata.groupby('homeTeamAbbr').size()
df

In [ ]:
df_gamedata.groupby('season').size()

In [ ]:
#月ごとの試合数
# convert integers to date 
df_gamedata["gameDate"] = pd.to_datetime(df_gamedata["gameDate"])
# extract month
df_gamedata['Year'] = df_gamedata["gameDate"].dt.month_name()
# extract days
df_gamedata['Week'] = df_gamedata["gameDate"].dt.day_name()

sns.set_style("whitegrid")
plt.figure(figsize=(8, 6))
ax = plt.gca()

sns.countplot(x='season', data=df_gamedata, hue='Year', lw=2, ax=ax).set(title='Games in Year, Season')
ax.legend(loc='center right', bbox_to_anchor=(1.5, 0.5), ncol=1)
plt.show()

### 【Plays Data】

プレイデータ：plays.csvファイルには、各ゲームのプレイレベル情報が含まれています。キー変数があるgameIdとplayId。
データを再生する
* gameId：ゲーム識別子、一意（数値）
* playId：ゲーム間で一意ではなく、識別子を再生します（数値）
* playDescription：遊びの説明（テキスト）
* quarter：ゲームクォーター（数値）
* down：ダウン（数値）
* yardsToGo：最初のダウンに必要な距離（数値）
* possessionTeam：チームのパント、プレースキック、またはボールのキックオフ（テキスト）
* specialTeamsPlayType：プレーの形成：エクストラポイント、フィールドゴール、キックオフまたはパント（テキスト）
* specialTeamsPlayResult：プレーの種類に応じたプレーの特別チームの結果：ブロックされたキックの試み、ブロックされたパント、ダウン、フェアキャッチ、キックの試みは良い、キックの試みは良くない、キックオフチームの回復、マフ、非特別なチームの結果、範囲外、リターンまたはタッチバック（テキスト）
* kickerId：プレー中のキッカー、パンター、またはキックオフスペシャリストのnflId（数値）
* returnerId：特別なチームが戻ってきた場合、プレー中のリターナーのnflId。劇中の複数のリターナーは;（テキスト）で区切られます
* kickBlockerId：ブロックされたフィールドゴールまたはブロックされたパントがあった場合のキックオンプレーのブロッカーのnflId（数値）
* yardlineSide：スクリメージライン（テキスト）に対応する3文字のチームコード
* yardlineNumber：スクリメージラインのヤードライン（数値）
* gameClock：プレイ時間（MM：SS）
* penaltyCodes：プレー中に発生したペナルティのNFL分類。標準のペナルティコードとそれに続くd手段は、ペナルティが防御にあったことを意味します。演劇に対する複数のペナルティは;（テキスト）で区切られます
* penaltyJerseyNumber：各ペナルティを犯したプレーヤーのジャージ番号とチームコード。演劇に対する複数のペナルティは;（テキスト）で区切られます
* penaltyYards：ペナルティによってpossitionTeamによって獲得されたヤード（数値）
* preSnapHomeScore：プレー前のホームスコア（数値）
* preSnapVisitorScore：プレー前の訪問チームのスコア（数値）
* passResult：specialTeamsPlayResult「非特別チーム結果」の場合のプレーのスクリメージ結果（C：完全パス、I：不完全パス、S：クォーターバックサック、IN：傍受パス、R：スクランブル、' '：デザインラッシュ、テキスト）
* kickLength：キックオフ、フィールドゴールまたはパントの空中でのキックの長さ（数値）
* kickReturnYardage：キックオフまたはパントでリターンがあった場合にリターンチームが獲得したヤード（数値）
* playResult：ペナルティヤード（数値）を含む、キックチームが獲得したネットヤード
* absoluteYardlineNumber：追跡データ座標におけるボールダウンフィールドの位置（数値）

In [ ]:
df_plays=pd.read_csv('../input/nfl-big-data-bowl-2022/plays.csv')
df_plays.head()

### 【PFFScoutingData】

In [ ]:
df_scouting=pd.read_csv('../input/nfl-big-data-bowl-2022/PFFScoutingData.csv')
df_scouting.head()

#### ■search correlated columns in PFFScoutingData.csv

In [ ]:
df_scouting.corr()

In [ ]:
pp_scouting = sns.pairplot(df_scouting)
plt.show()

### ■search correlated columns in Plays.csv

In [ ]:
df_plays.corr()

In [ ]:
pp_plays = sns.pairplot(df_plays)
plt.show()

### 【Tracking Data】

追跡データ：ファイルにtracking[season].csvは、シーズンのプレーヤー追跡データが含まれています[season]。主要変数でありgameId、playIdとnflId。
追跡データ ファイルにtracking[season].csvは、シーズンのプレーヤー追跡データが含まれています[season]。

* time：プレイのタイムスタンプ（時間、yyyy-mm-dd、hh：mm：ss）
* x：フィールドの長軸に沿ったプレーヤーの位置、0〜120ヤード。以下の図1を参照してください。（数値）
* y：フィールドの短軸に沿ったプレーヤーの位置、0〜53.3ヤード。以下の図1を参照してください。（数値）
* s：ヤード/秒単位の速度（数値）
* a：ヤード/秒での速度^ 2（数値）
* dis：前の時点からの移動距離（ヤード（数値））
* o：プレーヤーの向き（度）、0〜360度（数値）
* dir：プレーヤーの動きの角度（度）、0〜360度（数値）
* event：ボールスナップの瞬間、パスリリース、パスキャッチ、タックルなどのタグ付きプレイの詳細（テキスト）
* nflId：プレーヤー間で一意のプレーヤー識別番号（数値）
* displayName：プレイヤー名（テキスト）
* jerseyNumber：ジャージープレーヤー数（数値）
* position：プレイヤーポジショングループ（テキスト）
* team：対応するプレーヤーのチーム（アウェイまたはホーム）（テキスト）
* frameId：1から始まる各プレイのフレーム識別子（数値）
* gameId：ゲーム識別子、一意（数値）
* playId：ゲーム間で一意ではなく、識別子を再生します（数値）
* playDirection：オフェンスが動いている方向（左または右）

In [ ]:
df_t2018=pd.read_csv('../input/nfl-big-data-bowl-2022/tracking2018.csv')

In [ ]:
df_t2018.head()

In [ ]:
from PIL import Image
from skimage import io
f='https://upload.wikimedia.org/wikipedia/commons/thumb/c/c5/AmFBfield.svg/1200px-AmFBfield.svg.png'
a = io.imread(f)
# determining the height ratio
# resize image and save
from PIL import Image
from matplotlib import cm
import matplotlib.pyplot as plt
im = Image.fromarray(a)
im = im.resize((120,53))
plt.imshow(im)
im.save('sompic.png') # Saving the resized background image

In [ ]:
t=df_t2018.loc[(df_t2018['time'].str.contains('2018-12-30',case=False))]

In [ ]:
t.head(10)

In [ ]:
import time
from datetime import datetime, date, time, timedelta
t['time'][len(t)-1]
time_string = "2018-12-30 21:25:32.200"
format_string = "%Y-%m-%d %H:%M:%S.%f"

for i in range(100):
    date_object = datetime.strptime(time_string, format_string)
    date_object = date_object + timedelta(milliseconds=100)
    time_string=str(date_object)
    if '.' in time_string:
        time_string=time_string[:-3]
    else:
        time_string=time_string+'.000'
    final_string=time_string.split(' ')
    final_string=final_string[0]+'T'+final_string[1]
    print(final_string)
    break
print(len(t['time'].unique()))
z=t.loc[(t['time'].str.contains('2018-12-30T21:25:32.200')) & (t['gameId']==2018123000)]
z.index = range(len(z))
plt.rcParams["figure.figsize"] = [7.50, 3.50]
plt.rcParams["figure.autolayout"] = True
f='sompic.png'
a = io.imread(f)
#plt.imshow(a)
fig=plt.figure()
ax = fig.add_subplot(111)
a = io.imread(f)
print(len(z))
for i in range(len(z)):
    ax.imshow(a)
    if z[i:i+1]['team'][i]=='home': 
        ax.plot(z[i:i+1]['x'][i],z[i:i+1]['y'][i],'ro',color='blue')
    elif z[i:i+1]['team'][i]=='away':
        ax.plot(z[i:i+1]['x'][i],z[i:i+1]['y'][i],'ro',color='red')
    else:
        ax.plot(z[i:i+1]['x'][i],z[i:i+1]['y'][i],'ro',color='yellow')
ax.set_xlim(0, 120)
ax.set_ylim(0, 53.3)
ax.axis('on')
leg = ax.legend(loc='upper center')
plt.show()

In [ ]:
df2018=df_t2018.query('gameId==2018123000')
df2018.groupby('displayName').size().sort_values(ascending=True) \
    .plot(kind='barh', figsize=(10, 30
                               ),
         title='Participation time')
plt.show()

### ■Let's see only one person

In [ ]:
#1人だけみてみよう
track2018=df_t2018.query('playId==36 and displayName=="Justin Tucker"')
track2018[0:3]

In [ ]:
df_plays=pd.read_csv('../input/nfl-big-data-bowl-2022/plays.csv')
df_plays.head()

### 【Player Data】

プレーヤーデータ：このplayers.csvファイルには、追跡データファイルのいずれかに参加したプレーヤーからのプレーヤーレベルの情報が含まれています。キー変数ですnflId。
プレーヤーデータ
* gameId：ゲーム識別子、一意（数値）
* playId：ゲーム間で一意ではなく、識別子を再生します（数値）
* nflId：プレーヤー間で一意のプレーヤー識別番号（数値）
* Height：プレーヤーの身長（テキスト）
* Weight：プレーヤーの体重（数値）
* birthDate：生年月日（YYYY-MM-DD）
* collegeName：プレイヤーカレッジ（テキスト）
* Position：プレイヤーの位置（テキスト）
* displayName：プレイヤー名（テキスト）

In [ ]:
df_players=pd.read_csv('../input/nfl-big-data-bowl-2022/players.csv')
df_players.head()

### ■search correlated columns in players.csv

In [ ]:
df_players.corr()

In [ ]:
pp_players = sns.pairplot(df_players)
plt.show()

### Merging files 

In [ ]:
player_plays = pd.merge(df_players, df_plays, left_on='nflId', right_on='kickerId')
player_plays.head()


### ■Correlation　player.csv and plays.csv

In [ ]:
player_plays.corr()


In [ ]:
collegs_data = df_players.groupby('collegeName').size()


### ■Team size

In [ ]:
#チームの規模
mpl.rcParams['font.size']=12                 
mpl.rcParams['savefig.dpi']=100             
mpl.rcParams['figure.subplot.bottom']=.1 

stopwords = set(STOPWORDS)

wordcloud = WordCloud(
                          background_color='white',
                          stopwords=stopwords,
                          max_words=400,
                          max_font_size=40, 
                          random_state=42
                         ).generate(str(df_players['collegeName']))

print(wordcloud)
fig = plt.figure(1)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()
fig.savefig("word1.png", dpi=900)

In [ ]:
df_players['collegeName'].value_counts() \
    .sort_values(ascending=True) \
    .plot(kind='barh', figsize=(10, 60
                               ),
         title='Count of Players by Position')
plt.show()

### ■Players Position Count

In [ ]:
#どのポジションが多いかグラフで見てみよう
df_players['Position'].value_counts() \
    .sort_values(ascending=True) \
    .plot(kind='barh', figsize=(10, 15                               ),
         title='Count of Players by Position')
plt.show()

In [ ]:
len(df_players)

In [ ]:
df_players.query('collegeName=="James Madison"')

In [ ]:
df_gamedata.info()

In [ ]:
df_plays[df_plays['gameId']==2018090600].head()

In [ ]:
df_t2018.head()

In [ ]:
df_t2018[df_t2018['displayName']=='Justin Tucker']

#### ■PlayId36 player movement

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
df_t2018.query('gameId == 2018123000 and playId == 36').groupby('team') \
    .plot(x='x', y='y', ax=ax, style='.')
plt.legend().remove();
